# Worksheet 05

Name:  Isaac Chan
UID: U93997706

### Topics

- Cost Functions
- Kmeans

### Cost Function

Solving Data Science problems often starts by defining a metric with which to evaluate solutions were you able to find some. This metric is called a cost function. Data Science then backtracks and tries to find a process / algorithm to find solutions that can optimize for that cost function.

For example suppose you are asked to cluster three points A, B, C into two non-empty clusters. If someone gave you the solution `{A, B}, {C}`, how would you evaluate that this is a good solution?

Notice that because the clusters need to be non-empty and all points must be assigned to a cluster, it must be that two of the three points will be together in one cluster and the third will be alone in the other cluster.

In the above solution, if A and B are closer than A and C, and B and C, then this is a good solution. The smaller the distance between the two points in the same cluster (here A and B), the better the solution. So we can define our cost function to be that distance (between A and B here)!

The algorithm / process would involve clustering together the two closest points and put the third in its own cluster. This process optimizes for that cost function because no other pair of points could have a lower distance (although it could equal it).

### K means

a) (1-dimensional clustering) Walk through Lloyd's algorithm step by step on the following dataset:

`[0, .5, 1.5, 2, 6, 6.5, 7]` (note: each of these are 1-dimensional data points)

Given the initial centroids:

`[0, 2]`

1. Assign each data point to the nearest centroid:
    assignments: c1, c1, c2, c2, c2, c2, c2
2. Calculate the new centroids by finding the average of all the data points assigned to each centroid:
    c1 = 0.25   c2 = 4.6
3. Repeat assignments with new centroids:
    assignments: c1, c1, c1, c1, c2, c2, c2,
4. Calculate new centroids:
    c1 = 1      c2 = 6.5
5. Check for convergence:
    If the centroids do not change (or the change is below a certain threshold), the algorithm has converged. In this case, the centroids have changed, so we would repeat the assignment and update steps. However, if we continue the iterations, we'll find that the centroids stabilize at 1 and 6.5.
    So final centroids are 1 and 6.5 and final assignments are c1, c1, c1, c1, c2, c2, c2.

b) Describe in plain english what the cost function for k means is.

The cost function for KMeans measures how spread out the data points are within each cluster. It calculates the tightness of the data points in each cluster.

c) For the same number of clusters K, why could there be very different solutions to the K means algorithm on a given dataset?

The KMeans algorithm can produce different solutions on the same dataset due to its sensitivity to initial centroid placements and the possibility of converging to local minima. Other factors like data order, distance metrics, and the presence of noise or outliers can also influence the clustering outcome.

d) Does Lloyd's Algorithm always converge? Why / why not?

Yes, Lloyd's Algorithm always converges, but it might converge to a local minimum rather than a global minimum. This is because each iteration of the algorithm reduces the cost function, ensuring no infinite loops, but the final solution can be dependent on the initial centroid placements.

e) Follow along in class the implementation of Kmeans

In [ ]:
import numpy as np
from PIL import Image as im
import matplotlib.pyplot as plt
import sklearn.datasets as datasets

centers = [[0, 0], [2, 2], [-3, 2], [2, -4]]
X, _ = datasets.make_blobs(n_samples=300, centers=centers, cluster_std=1, random_state=0)

class KMeans():

    def __init__(self, data, k):
        self.data = data
        self.k = k
        self.assignment = [-1 for _ in range(len(data))]
        self.snaps = []
    
    def snap(self, centers):
        TEMPFILE = "temp.png"

        fig, ax = plt.subplots()
        ax.scatter(X[:, 0], X[:, 1], c=self.assignment)
        ax.scatter(centers[:,0], centers[:, 1], c='r')
        fig.savefig(TEMPFILE)
        plt.close()
        self.snaps.append(im.fromarray(np.asarray(im.open(TEMPFILE))))


    def lloyds(self):
        # Randomly initialize the centroids
        centers = self.data[np.random.choice(self.data.shape[0], self.k, replace=False)]
        
        while True:
            # Assignment step
            distances = np.linalg.norm(self.data[:, np.newaxis] - centers, axis=2)
            new_assignment = np.argmin(distances, axis=1)
            
            # Check for convergence
            if np.array_equal(new_assignment, self.assignment):
                break

            self.assignment = new_assignment

            # Update step
            new_centers = np.array([self.data[self.assignment == i].mean(axis=0) for i in range(self.k)])

            # Take a snapshot for visualization
            self.snap(new_centers)

            # Check for convergence
            if np.array_equal(new_centers, centers):
                break

            centers = new_centers

            

kmeans = KMeans(X, 6)
kmeans.lloyds()
images = kmeans.snaps

images[0].save(
    'kmeans.gif',
    optimize=False,
    save_all=True,
    append_images=images[1:],
    loop=0,
    duration=500
)